In [1]:
#import packages.
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
#tell computer where to get data
filepath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
#creates a data set all in lower case. 
text = open(filepath,'rb').read().decode(encoding = 'utf-8').lower()

In [4]:
#sort characters
characters = sorted(set(text))

In [5]:
#creates a library that has characters as a key and index as value for all indices and character which assigns a number to each character.
char_to_index = dict((c,i) for i, c in enumerate(characters))
index_to_char = dict((i,c) for i, c in enumerate(characters))

In [6]:
#tells computer how long a sequence should be and where the next stest should start.
SEQ_LENGTH = 40
STEP_SIZE = 3

In [7]:
#creates empty lists. 
sentences = []
next_characters = []

In [8]:
for i in range(0, len(text) -SEQ_LENGTH,STEP_SIZE):
    sentences.append(text[i:i+SEQ_LENGTH]) #if sequence is 5 we get characters up until 4
    next_characters.append(text[i+SEQ_LENGTH])#gets next correct letters
#this code fills the two lists

In [9]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype = np.bool_)
y = np.zeros((len(sentences), len(characters)), dtype = np.bool_)

In [10]:
for i, sentence in enumerate (sentences):
    for t, character in enumerate (sentence):
        x[i,t, char_to_index[character]] = 1
    y[i,char_to_index[next_characters[i]]] = 1

In [11]:
model = Sequential()
model.add(LSTM(128,input_shape = (SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))#scales values

In [12]:
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.01))

In [13]:
model.fit(x,y, batch_size =256, epochs = 20)

Epoch 1/20
1453/1453 [==============================] - 79s 54ms/step - loss: 2.4593
Epoch 2/20
1453/1453 [==============================] - 87s 60ms/step - loss: 2.1066
Epoch 3/20
1453/1453 [==============================] - 88s 60ms/step - loss: 1.9725
Epoch 4/20
1453/1453 [==============================] - 87s 60ms/step - loss: 1.8824
Epoch 5/20
1453/1453 [==============================] - 86s 59ms/step - loss: 1.8158
Epoch 6/20
1453/1453 [==============================] - 87s 60ms/step - loss: 1.7618
Epoch 7/20
1453/1453 [==============================] - 86s 59ms/step - loss: 1.7168
Epoch 8/20
1453/1453 [==============================] - 86s 60ms/step - loss: 1.6792
Epoch 9/20
1453/1453 [==============================] - 87s 60ms/step - loss: 1.6466
Epoch 10/20
1453/1453 [==============================] - 87s 60ms/step - loss: 1.6189
Epoch 11/20
1453/1453 [==============================] - 88s 61ms/step - loss: 1.5941
Epoch 12/20
1453/1453 [==============================] - 88s 61

In [14]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


In [15]:
model = tf.keras.models.load_model('textgenerator.model')

In [16]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [20]:
print('--------0.2----------')
print(generate_text(300, 0.2))
print('--------0.4----------')
print(generate_text(300, 0.4))
print('--------0.6---------')
print(generate_text(300, 0.6))
print('--------0.8----------')
print(generate_text(300, 0.8))
print ('--------1----------')
print(generate_text(300, 1.0))

--------0.2----------
's broken wing,
redeem from broking pawnest the see the forth,
and there is the will speak the forth the seeth
the man and from him to the father the man
that was do it breath, and the world be the partor,
and the world comes the heads of the face
that i shall be the mind of her fair the heart
that the king of the face of the man be the f
--------0.4----------
aris to saint peter's church,
or i will not be the mother with her comes,
and the father of the begin the was fair,
there is the will of the provow'st the mark,
she is mers of his marries the fear,
and there is the company the more for hourself
and discoused the man of words and fair tongue;
for there is no more the deed for the great,
an
--------0.6---------
and dion, have
been both at delphos, and the wrengen thee:
you so, all the paint of the man upon the gates
i shall be more can the world resore the deeds
the shill to make upon the frathor of when
no more of the giver with up, and i king recemper
that u